**Quick notebook to reverse geocode adddresses for all U.S. counties (fips as of 2015)**
Note: Since this is purely to get a geographic centroid for address identification and U.S. Census updates to county fips codes involved mostly name changes as opposed to geographic changes, ok to use old data.

In [1]:
#re-imports modules that have changed
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [37]:
import os
import logging
import pandas as pd
import requests


In [27]:
# Load data
filepath = "../data/Gaz_counties_national.txt"
data = pd.read_csv(filepath, encoding="latin-1", sep='\t')

In [28]:
# Process data
data.columns = data.columns.str.strip()
data['GEOID'] = data['GEOID'].astype(str).str.zfill(5)
data = data[['USPS', 'GEOID', 'NAME', 'INTPTLAT', 'INTPTLONG']].copy()
data.columns = ['state_abbr', 'fips', 'name', 'lat', 'long']

In [50]:
geo=ReverseGeocode()
addresses = []
for index, row in data[0:3].iterrows():
    formats = []
    lat = row['lat']
    long = row['long']
    response = geo.reverse_geocode(lat, long)
    for item in response['results']: 
        address = item['formatted_address']
        formats.append(address)
    addresses.append(formats)

In [40]:
class ReverseGeocode(): 
    """
    Fetchs a address and geocoding data from Google Geocoding API. 
    https://developers.google.com/maps/documentation/geocoding/intro#ReverseGeocoding 
    https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
    Takes the lat, long as geo points.
    Returns the response as a json object.
    """
    
    def __init__(self):
        self._url = "https://maps.googleapis.com/maps/api/geocode/json?"
        self._api_key = os.environ['GOOGLE_GEOCODING_API_KEY']
    
    def reverse_geocode(self, lat, long):
        """
        Make a call to the api to return election info
        """
        payload = {
            "latlng": f"{lat},{long}",
            "key": self._api_key
        } 
        response = requests.get(self._url, params=payload)
        try: 
            response.raise_for_status() 
            return response.json()
        except requests.exceptions.HTTPError as error:
            # Error in request 
            logging.error(error)
        except requests.exceptions.RequestException as error:
            # Catastrophic error 
            logging.error(error)
            raise

In [51]:
addresses

[['889 Love Ln, Prattville, AL 36067, USA',
  '887 W D Dr, Prattville, AL 36067, USA',
  '889 Love Ln, Prattville, AL 36067, USA',
  'Love Ln, Prattville, AL 36067, USA',
  'Prattville, AL 36067, USA',
  'Autauga County, AL, USA',
  'Alabama, USA',
  'United States'],
 ['Unnamed Road, Loxley, AL 36551, USA',
  'Loxley, AL 36551, USA',
  'Baldwin County, AL, USA',
  'Alabama, USA',
  'United States'],
 ['94 Stevens Rd, Clayton, AL 36016, USA',
  'Saw Mill Rd, Clayton, AL 36016, USA',
  'Clayton, AL 36016, USA',
  'Barbour County, AL, USA',
  'Alabama, USA',
  'United States']]

In [42]:
response = geo.reverse_geocode(lat, long)

In [46]:
len(response['results'])

5

In [47]:
response['results'][0]

{'address_components': [{'long_name': 'Unnamed Road',
   'short_name': 'Unnamed Road',
   'types': ['route']},
  {'long_name': 'Yauco',
   'short_name': 'Yauco',
   'types': ['locality', 'political']},
  {'long_name': 'Algarrobo',
   'short_name': 'Algarrobo',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Yauco',
   'short_name': 'Yauco',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Puerto Rico',
   'short_name': 'PR',
   'types': ['country', 'political']},
  {'long_name': '00698', 'short_name': '00698', 'types': ['postal_code']}],
 'formatted_address': 'Unnamed Road, Yauco, 00698, Puerto Rico',
 'geometry': {'bounds': {'northeast': {'lat': 18.0857539, 'lng': -66.858121},
   'southwest': {'lat': 18.08351, 'lng': -66.86131739999999}},
  'location': {'lat': 18.0841117, 'lng': -66.8599356},
  'location_type': 'GEOMETRIC_CENTER',
  'viewport': {'northeast': {'lat': 18.0859809302915, 'lng': -66.858121},
   'southwest': {'lat': 18.

In [49]:
for item in response['results']: 
    address = item['formatted_address']
    print(address)

Unnamed Road, Yauco, 00698, Puerto Rico
Algarrobo, Yauco 00698, Puerto Rico
Yauco, 00698, Puerto Rico
Yauco, Puerto Rico
Puerto Rico
